In [1]:
def wrangle_grades():
    """
    Read student_grades csv file into a pandas DataFrame,
    drop student_id column, replace whitespaces with NaN values,
    drop any rows with Null values, convert all columns to int64,
    return cleaned student grades DataFrame.
    """
    # Acquire data from csv file.
    grades = pd.read_csv("student_grades.csv")

    # Replace white space values with NaN values.
    grades = grades.replace(r"^\s*$", np.nan, regex=True)

    # Drop all rows with NaN values.
    df = grades.dropna()

    # Convert all columns to int64 data types.
    df = df.astype("int")

    return df


In [4]:
 grades = pd.read_csv("student_grades.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'student_grades.csv'

# Feature Engineering

For some definitions, handling outliers and missing values, scaling, and encoding may be considered feature engineering. Here we'll draw a distinction between data preparation, data preprocessing, and feature engineering.

- **data preparation**: the basic data cleaning necessary to get our data ready for exploration/analysis, e.g. correcting data types, fixing typos
- **data preprocessing**: further data transformation done for the sake of modeling, as oppsoed to exploration/analysis, e.g. scaling, imputing, encoding
- **feature engineering**: adding, combining, or removing features; usually with the help of domain knowledge

Feature engineering can happen as part of data exploration or modeling, and engineered featured are also commonly explored.

Some examples of feature engineering by this definition:

- domain-based conversion (example: farenheit to celsius, BMI calculation, log transformation)
- domain based cutoffs (example: age >= 18 = is_adult; also dates)

- add / subtract (example: zillow dataset: beds + baths = room_count; total_sqft - 200 * bedrooms - 40 * bathrooms = living_area)
- combine as booleans as a count (example: telco_churn: streaming + backups + ...  = service_count)
- multiply / divide (example: tips dataset: total_bill / size = price_per_person)
- ratios (example: tips dataset: tip / total_bill = tip percentage)

Simplify!

- categorical with many unique values to top 3 + "Other"
- categorical to boolean: pool count -> has pool
- continous -> categorical via binning (aka quantization or discretization) (example: income -> high, medium, low earner)

In this lesson we'll cover some *automated* **feature selection** methods, that is, methods for determining which features are the most important.

- SelectKBest
- Recursive Feature Elimination
- Sequential Feature Selection

In [ ]:
mycols = ['col1', 'col2', 'col3', 'col4']

In [ ]:
# # start with initializing empty lists
# # feature space will contain our features
# feature_space = []
# # scores will contain model performance
# scores = []
# # iterate through the list of columns:
# for col in mycols:
#     # add a new feature in each cycle
#     feature_space.append(col)
#     # fit a model based on the new feature space
#     model_object.fit(X[feature_space], y)
#     # get the score for that model
#     scores.append(model_object.score(X[feature_space], y))
    
# my_scores = dict(zip(feature_space, scores))

## Setup

In [3]:
import numpy as np
import pandas as pd
import wrangle
from sklearn.feature_selection import SelectKBest, RFE, f_regression, SequentialFeatureSelector

In [ ]:
df = wrangle_grades()

In [ ]:
# train_validate, test = train_test_split(df)
# train, validate = train_test_split(train_validate)

train, validate, test = wrangle.split_continuous(df)

In [ ]:
train.head(3)

In [ ]:
X_train, y_train = train[['exam1', 'exam2', 'exam3']], train.final_grade
X_validate, y_validate = validate[['exam1', 'exam2', 'exam3']], validate.final_grade
X_test, y_test = test[['exam1', 'exam2', 'exam3']], test.final_grade

## Select K Best

- looks at each feature in isolation against the target based on correlation
- fastest of all approaches covered in this lesson
- doesn't consider feature interactions
- After fitting: `.scores_`, `.pvalues_`, `.get_support()`, and `.transform`

In [ ]:
# make the thing
kbest = SelectKBest(f_regression, k=2)
# fit the thing
_ = kbest.fit(X_train, y_train)
# statistical f-value:
kbest.scores_
#p value: 
kbest.pvalues_
kbest_results = pd.DataFrame(
    dict(p=kbest.pvalues_, f=kbest.scores_),
                             index = X_train.columns)
kbest_results
# get-support() will output a boolean mask to tell me which features were selected
# we can apply this mask to the columns in our original dataframe
X_train.columns[kbest.get_support()]
# kbest transform will convert our information to the selected feature subspace
# ****buuuuuut, its just a numpy array
kbest.transform(X_train)[:5]
X_train_transformed = pd.DataFrame(
    kbest.transform(X_train),
    columns=X_train.columns[kbest.get_support()],
    index=X_train.index
)
X_train_transformed.head(3)

In [ ]:
# make the thing
kbest = SelectKBest(f_regression, k=2)
# fit the thing
_ = kbest.fit(X_train, y_train)

In [ ]:
# statistical f-value:
kbest.scores_
#p value: 
kbest.pvalues_

In [ ]:
kbest_results = pd.DataFrame(
    dict(p=kbest.pvalues_, f=kbest.scores_),
                             index = X_train.columns)

In [ ]:
kbest_results

In [ ]:
# get-support() will output a boolean mask to tell me which features were selected
# we can apply this mask to the columns in our original dataframe
X_train.columns[kbest.get_support()]

In [ ]:
# kbest transform will convert our information to the selected feature subspace
# ****buuuuuut, its just a numpy array
kbest.transform(X_train)[:5]

In [ ]:
X_train_transformed = pd.DataFrame(
    kbest.transform(X_train),
    columns=X_train.columns[kbest.get_support()],
    index=X_train.index
)

In [ ]:
X_train_transformed.head(3)

## RFE

- Recursive Feature Elimination
- Progressively eliminate features based on importance to the model
- Requires a model with either a `.coef_` or `.feature_importances_` property
- After fitting: `.ranking_`, `.get_support()`, and `.transform()`

In [ ]:
# make a model object to use in RFE process.
# The model is here to give us metrics on feature importance and model score
# allowing us to recursively reduce the number of features to reach our desired space
model = LinearRegression()

# make thing
rfe = RFE(model, n_features_to_select=2)
# fit thing
rfe.fit(X_train, y_train)

rfe.ranking_

pd.DataFrame(
{
    'rfe_ranking': rfe.ranking_
},index = X_train.columns)

rfe.get_support()

X_train_transformed = pd.DataFrame(
rfe.transform(X_train),
index = X_train.index,
columns = X_train.columns[rfe.support_])

X_train_transformed.head()

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
# make a model object to use in RFE process.
# The model is here to give us metrics on feature importance and model score
# allowing us to recursively reduce the number of features to reach our desired space
model = LinearRegression()

In [ ]:
# make thing
rfe = RFE(model, n_features_to_select=2)
# fit thing
rfe.fit(X_train, y_train)

In [ ]:
rfe.ranking_

In [ ]:
pd.DataFrame(
{
    'rfe_ranking': rfe.ranking_
},index = X_train.columns)

In [ ]:
rfe.get_support()

In [ ]:
X_train_transformed = pd.DataFrame(
rfe.transform(X_train),
index = X_train.index,
columns = X_train.columns[rfe.support_])

In [ ]:
X_train_transformed.head()

## Sequential Feature Selector

- progressively adds features based on cross validated model performance
- forwards: start with 0, add the best additional feature until you have the desired number
- backwards: start with all features, remove the worst performing until you have the desired number
- After fitting: `.support_`, `.transform`

In [ ]:
model = LinearRegression()
sfs = SequentialFeatureSelector(model, n_features_to_select=2)
sfs.fit(X_train, y_train)

In [ ]:
X_train_transformed = pd.DataFrame(
sfs.transform(X_train),
index = X_train.index,
columns = X_train.columns[sfs.support_])

In [ ]:
X_train_transformed

## Conclusion

- Simpler models handle change + variability better
- Use RFE to narrow down your features and find the best ones, if your dataset is large (> 1GB; `df.info()`) use select k best instead
- Remember: feature engineering is much more than feature selection!